# ImpPres LLM Baseline

You have to implement in this notebook a baseline for ImpPres classification using an LLM.
This baseline must be implemented using DSPy.



In [111]:
# Configure the DSPy environment with the language model - for grok the parameters must be:
# env variable should be in os.environ['XAI_API_KEY']
# "xai/grok-3-mini"
from dotenv import load_dotenv
import os
import dspy
load_dotenv("grok_key.ini") 
lm = dspy.LM('xai/grok-3-mini', api_key=os.environ['XAI_API_KEY'])



# for ollama 
# lm = dspy.LM('ollama_chat/devstral', api_base='http://localhost:11434', api_key='')
dspy.configure(lm=lm)

In [94]:
from typing import Literal
from tqdm import tqdm
import dspy

# Signature for the NLI task
class NLISignature(dspy.Signature):
    """
    Classify the relationship between the premise and hypothesis 
    to a label: entailment, neutral or contradiction.
    """
    premise: str = dspy.InputField()
    hypothesis: str = dspy.InputField()
    label: Literal['entailment', 'neutral', 'contradiction'] = dspy.OutputField()

# A class for Parallel processing with progress display
class NLIClassifier(dspy.Module):
    def __init__(self, predictor_module: dspy.Module, batch_size: int = 15, num_threads: int = 6):
        super().__init__()
        self.predictor = predictor_module  # Predict, ChainOfThought, etc.
        self.batch_size = batch_size
        self.num_threads = num_threads

    def forward(self, examples: dspy.Example) -> list[dspy.Prediction]:
        # Display progress with tqdm while processing
        results = []
        for i in tqdm(range(0, len(examples), self.batch_size), desc="Processing"):
            sub_batch = examples[i:i + self.batch_size]
            processed = self.predictor.batch( # perform batch processing
                sub_batch,
                num_threads=self.num_threads
            )
            results.extend(processed)

        return results


## Load ImpPres dataset

In [95]:
from datasets import load_dataset

sections = ['presupposition_all_n_presupposition', 
            'presupposition_both_presupposition', 
            'presupposition_change_of_state', 
            'presupposition_cleft_existence', 
            'presupposition_cleft_uniqueness', 
            'presupposition_only_presupposition', 
            'presupposition_possessed_definites_existence', 
            'presupposition_possessed_definites_uniqueness', 
            'presupposition_question_presupposition']

dataset = {}
for section in sections:
    print(f"Loading dataset for section: {section}")
    dataset[section] = load_dataset("facebook/imppres", section)

Loading dataset for section: presupposition_all_n_presupposition
Loading dataset for section: presupposition_both_presupposition
Loading dataset for section: presupposition_change_of_state
Loading dataset for section: presupposition_cleft_existence
Loading dataset for section: presupposition_cleft_uniqueness
Loading dataset for section: presupposition_only_presupposition
Loading dataset for section: presupposition_possessed_definites_existence
Loading dataset for section: presupposition_possessed_definites_uniqueness
Loading dataset for section: presupposition_question_presupposition


In [96]:
dataset

{'presupposition_all_n_presupposition': DatasetDict({
     all_n_presupposition: Dataset({
         features: ['premise', 'hypothesis', 'trigger', 'trigger1', 'trigger2', 'presupposition', 'gold_label', 'UID', 'pairID', 'paradigmID'],
         num_rows: 1900
     })
 }),
 'presupposition_both_presupposition': DatasetDict({
     both_presupposition: Dataset({
         features: ['premise', 'hypothesis', 'trigger', 'trigger1', 'trigger2', 'presupposition', 'gold_label', 'UID', 'pairID', 'paradigmID'],
         num_rows: 1900
     })
 }),
 'presupposition_change_of_state': DatasetDict({
     change_of_state: Dataset({
         features: ['premise', 'hypothesis', 'trigger', 'trigger1', 'trigger2', 'presupposition', 'gold_label', 'UID', 'pairID', 'paradigmID'],
         num_rows: 1900
     })
 }),
 'presupposition_cleft_existence': DatasetDict({
     cleft_existence: Dataset({
         features: ['premise', 'hypothesis', 'trigger', 'trigger1', 'trigger2', 'presupposition', 'gold_label', 'UI

## Evaluate Metrics

Let's use the huggingface `evaluate` package to compute the performance of the baseline.


In [97]:
import evaluate
clf_metrics = evaluate.combine(["accuracy", "f1", "precision", "recall"])

## Your Turn

Compute the classification metrics on the baseline LLM model on each test section of the ANLI dataset for samples that have a non-empty 'reason' field.

You also must show a comparison between the DeBERTa baseline model and this LLM baseline model. The comparison metric should compute the agreement between the two models:
* On how many samples they are both correct [Correct]
* On how many samples Model1 is correct and Model2 is incorrect [Correct1]
* On how many samples Model1 is incorrect and Model2 is correct [Correct2]
* On how many samples both are incorrect [Incorrect]

# IMPORTANT NOTE:

Following discussing with Michael - it was agreed that we use smaller sample than the whole impPres data set (17,000 samples) , as long as it is diverse.
So - we took 212 samples from each of the 9 data sets , summing up to 1908 examples.

In [98]:
# Organizing the dataset into examples

section_to_split = {
    'presupposition_all_n_presupposition': 'all_n_presupposition',
    'presupposition_both_presupposition': 'both_presupposition',
    'presupposition_change_of_state': 'change_of_state',
    'presupposition_cleft_existence': 'cleft_existence',
    'presupposition_cleft_uniqueness': 'cleft_uniqueness',
    'presupposition_only_presupposition': 'only_presupposition',
    'presupposition_possessed_definites_existence': 'possessed_definites_existence',
    'presupposition_possessed_definites_uniqueness': 'possessed_definites_uniqueness',
    'presupposition_question_presupposition': 'question_presupposition',
}

preprocessed_examples = [
    dspy.Example(
        premise=row["premise"],
        hypothesis=row["hypothesis"],
        label=row["gold_label"]
    ).with_inputs("premise", "hypothesis")
    for k, v in section_to_split.items()
    for row in dataset[k][v]
]

# use the first 212 examples for each section (in total ~1900 examples)
examples = []
for i in range(len(section_to_split)):
    start_index = i * 212
    end_index = start_index + 212
    examples.extend(preprocessed_examples[start_index:end_index])

print(f"Total examples: {len(examples)}")


# Orgnaizing the data sets set
# pick examples randomly for training
import random
train_set_size = 30 # tradeoff between quality and speed after testing
trainset = random.sample(examples[:len(examples)//2], train_set_size) # pick examples from the first half to avoid bias


# pick examples randomly for evaluation
eval_set_size = 60  # size of the evaluation set
evaluate_set = random.sample(examples[len(examples)//2:], eval_set_size)

# unrequired verbosity
import warnings
warnings.filterwarnings("ignore", category=FutureWarning, message=".*encoder_attention_mask.*")



Total examples: 1908


In [ ]:
from dspy.teleprompt import BootstrapFewShot, COPRO, KNNFewShot
from sentence_transformers import SentenceTransformer
import evaluate

# Load metrics
accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")
precision = evaluate.load("precision")
recall = evaluate.load("recall")

label2id = {"entailment": 0, "neutral": 1, "contradiction": 2}

def exact_match(example, pred, trace=None):
    # Ensure both labels are strings and lowercase
    ex_label = str(example.label).strip().lower()
    pred_label = str(pred.label).strip().lower()

    # In case example.label is already an int, use reverse mapping
    if ex_label.isdigit():
        id2label = {v: k for k, v in label2id.items()}
        ex_label = id2label[int(ex_label)]

    return label2id.get(pred_label) == label2id.get(ex_label)

def compute_metrics(preds, golds):
    return {
        "accuracy": accuracy.compute(predictions=preds, references=golds)["accuracy"],
        "precision": precision.compute(predictions=preds, references=golds, average="macro")["precision"],
        "recall": recall.compute(predictions=preds, references=golds, average="macro")["recall"],
        "f1": f1.compute(predictions=preds, references=golds, average="macro")["f1"],
    }

# Prepare models
model_simple = dspy.Predict(NLISignature)
model_cot = dspy.ChainOfThought(NLISignature)

# --- Optimization phase ---
# 1. BootstrapFewShot
bootstrap = BootstrapFewShot(metric=exact_match)
optimized_bootstrap = bootstrap.compile(student=model_simple, trainset=trainset)

# 2. COPRO on our cot model
copro = COPRO(metric=exact_match, max_trials=5, depth=2, breadth=3)
optimized_cot = copro.compile(student=model_cot, trainset=trainset,eval_kwargs={})

# # 3. KNNFewShot
# class EmbedderWrapper:
#     def __init__(self, model_name): self.model = SentenceTransformer(model_name)
#     def __call__(self, texts): return self.model.encode(texts, convert_to_numpy=True)

# vectorizer = EmbedderWrapper('all-MiniLM-L6-v2')
# knn = KNNFewShot(k=5, trainset=trainset, vectorizer=vectorizer)
# optimized_knn = knn.compile(student=model_simple)



 13%|█▎        | 4/30 [00:30<03:20,  7.70s/it]
2025/08/07 20:12:50 INFO dspy.teleprompt.copro_optimizer: Iteration Depth: 1/2.
2025/08/07 20:12:50 INFO dspy.teleprompt.copro_optimizer: At Depth 1/2, Evaluating Prompt Candidate #1/3 for Predictor 1 of 1.


Bootstrapped 4 full traces after 4 examples for up to 1 rounds, amounting to 4 attempts.


2025/08/07 20:13:13 INFO dspy.evaluate.evaluate: Average Metric: 27 / 30 (90.0%)
2025/08/07 20:13:13 INFO dspy.teleprompt.copro_optimizer: At Depth 1/2, Evaluating Prompt Candidate #2/3 for Predictor 1 of 1.
2025/08/07 20:13:39 INFO dspy.evaluate.evaluate: Average Metric: 29 / 30 (96.7%)
2025/08/07 20:13:39 INFO dspy.teleprompt.copro_optimizer: At Depth 1/2, Evaluating Prompt Candidate #3/3 for Predictor 1 of 1.
2025/08/07 20:14:05 INFO dspy.evaluate.evaluate: Average Metric: 28 / 30 (93.3%)
2025/08/07 20:14:12 INFO dspy.teleprompt.copro_optimizer: Iteration Depth: 2/2.
2025/08/07 20:14:12 INFO dspy.teleprompt.copro_optimizer: At Depth 2/2, Evaluating Prompt Candidate #1/3 for Predictor 1 of 1.
2025/08/07 20:14:39 INFO dspy.evaluate.evaluate: Average Metric: 27 / 30 (90.0%)
2025/08/07 20:14:39 INFO dspy.teleprompt.copro_optimizer: At Depth 2/2, Evaluating Prompt Candidate #2/3 for Predictor 1 of 1.
2025/08/07 20:15:06 INFO dspy.evaluate.evaluate: Average Metric: 28 / 30 (93.3%)
2025/08

In [104]:
import contextlib
import io

# --- Evaluation phase (Silent Mode) ---

print("📊 Evaluating Models... this might take a long time... adjust eval_set_size , train_set_size to speed up the process")

results = {}
silent_output = io.StringIO()
optimized_dict = {
    "BootstrapFewShot": optimized_bootstrap,
    "COPRO_CoT": optimized_cot,
    # "KNNFewShot": optimized_knn,
}

with contextlib.redirect_stdout(silent_output), contextlib.redirect_stderr(silent_output):
    for name, predictor in optimized_dict.items():
        program = NLIClassifier(predictor_module=predictor)
        predictions = program(evaluate_set) # "forward" method is called implicitly

        y_true = [ex.label for ex in evaluate_set[:len(predictions)]]
        y_pred = [label2id[pred.label.strip().lower()] for pred in predictions]

        results[name] = compute_metrics(y_pred, y_true)

# --- Display results ---
from pprint import pprint
print("📊 Model Evaluation Results:\n")
pprint(results)

📊 Evaluating Models... this might take a long time... adjust eval_set_size , train_set_size to speed up the process
📊 Model Evaluation Results:

{'BootstrapFewShot': {'accuracy': 0.9833333333333333,
                      'f1': 0.9808187134502924,
                      'precision': 0.9885057471264368,
                      'recall': 0.9743589743589745},
 'COPRO_CoT': {'accuracy': 0.9333333333333333,
               'f1': 0.9358730158730159,
               'precision': 0.9583333333333334,
               'recall': 0.921727395411606}}


In [105]:
# As you can see from the above output, BootstrapFewShot has the highest scores.
# reference output:

# {'BootstrapFewShot': {'accuracy': 0.9833333333333333,
#                       'f1': 0.9808187134502924,
#                       'precision': 0.9885057471264368,
#                       'recall': 0.9743589743589745},
#  'COPRO_CoT': {'accuracy': 0.9333333333333333,
#                'f1': 0.9358730158730159,
#                'precision': 0.9583333333333334,
#                'recall': 0.921727395411606}}


# The best performing model is optimized_bootstrap -> BootstrapFewShot ,  which automates the creation of effective few-shot demonstrations 

In [106]:
# run the best model on all of the examples , and report the results
best_model = optimized_bootstrap
program = NLIClassifier(predictor_module=best_model)
predictions = program.forward(examples)
y_true = [ex.label for ex in examples]
y_pred = [label2id[pred.label.strip().lower()] for pred in predictions]
# Compute final metrics
final_results = compute_metrics(y_pred, y_true)

Processing:   0%|          | 0/128 [00:00<?, ?it/s]

Processed 15 / 15 examples: 100%|██████████| 15/15 [00:19<00:00,  1.33s/it]

Processing:   1%|          | 1/128 [00:20<42:21, 20.01s/it]


Processed 15 / 15 examples: 100%|██████████| 15/15 [00:23<00:00,  1.58s/it]

Processing:   2%|▏         | 2/128 [00:43<46:37, 22.20s/it]

...

Processing:  98%|█████████▊| 126/128 [35:23<00:34, 17.45s/it]


Processed 15 / 15 examples: 100%|██████████| 15/15 [00:19<00:00,  1.29s/it]

Processing:  99%|█████████▉| 127/128 [35:42<00:18, 18.03s/it]


Processed 3 / 3 examples: 100%|██████████| 3/3 [00:05<00:00,  1.98s/it]

Processing: 100%|██████████| 128/128 [35:48<00:00, 16.78s/it]

# TASK 2.3

Compute the classification metrics on the baseline LLM model on each test section of the ANLI dataset for samples that have a non-empty 'reason' field.

In [109]:
print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")
print("\n")
print(" -> Final Evaluation Results on All Examples: <- \n"
      f"Accuracy: {final_results['accuracy']}, \n"
      f"F1: {final_results['f1']}, \n"
      f"Precision: {final_results['precision']}, \n"
      f"Recall: {final_results['recall']} \n")
print("\n")
print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@


 -> Final Evaluation Results on All Examples: <- 
Accuracy: 0.9858490566037735, 
F1: 0.985355433901831, 
Precision: 0.9889684783159963, 
Recall: 0.9823544187906642 



@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@


You also must show a comparison between the DeBERTa baseline model and this LLM baseline model. The comparison metric should compute the agreement between the two models:
* On how many samples they are both correct [Correct]
* On how many samples Model1 is correct and Model2 is incorrect [Correct1]
* On how many samples Model1 is incorrect and Model2 is correct [Correct2]
* On how many samples both are incorrect [Incorrect]

In [110]:
# how many samples they are both correct
%store -r DeBERTa_v3_predictions
optimized_llm_predictions = predictions
TEST_SIZE = len(optimized_llm_predictions)

non_llm_predictions = []
for i in range(len(section_to_split)):
    start_index = i * 212
    end_index = start_index + 212
    non_llm_predictions.extend(DeBERTa_v3_predictions[start_index:end_index])

# on how many samples both models are correct
correct_predictions = sum(
    1 for pred, llm_pred in zip(non_llm_predictions,optimized_llm_predictions)
    if (llm_pred.label == pred['gold_label']) and (pred['gold_label'] == pred['pred_label'])
)

print(f"Both models are correct on {correct_predictions} out of {TEST_SIZE} samples.")
print(f"Both models are correct on {correct_predictions / TEST_SIZE * 100:.2f}% of the samples.")
print("\n")

# On how many samples llm is correct and DeBERTa_v3_ is incorrect
llm_correct_deberta_incorrect = sum(
    1 for pred, llm_pred in zip(non_llm_predictions,optimized_llm_predictions)
    if (llm_pred.label == pred['gold_label']) and (pred['gold_label'] != pred['pred_label'])
)
print(f"LLM is correct and DeBERTa_v3 is incorrect on {llm_correct_deberta_incorrect} out of {TEST_SIZE} samples.")
print(f"LLM is correct and DeBERTa_v3 is incorrect on {llm_correct_deberta_incorrect / TEST_SIZE * 100:.2f}% of the samples.")
print("\n")

# On how many samples DeBERTa_v3 is correct and llm is incorrect
deberta_correct_llm_incorrect = sum(
    1 for pred, llm_pred in zip(non_llm_predictions,optimized_llm_predictions)
    if (pred['pred_label'] == pred['gold_label']) and (llm_pred.label != pred['gold_label'])
)
print(f"DeBERTa_v3 is correct and LLM is incorrect on {deberta_correct_llm_incorrect} out of {TEST_SIZE} samples.")
print(f"DeBERTa_v3 is correct and LLM is incorrect on {deberta_correct_llm_incorrect / TEST_SIZE * 100:.2f}% of the samples.")
print("\n")


# on how many samples both models are incorrect
both_incorrect = sum(
    1 for pred, llm_pred in zip(non_llm_predictions,optimized_llm_predictions)
    if (llm_pred.label != pred['gold_label']) and (pred['pred_label'] != pred['gold_label'])
)
print(f"Both models are incorrect on {both_incorrect} out of {TEST_SIZE} samples.")
print(f"Both models are incorrect on {both_incorrect / TEST_SIZE * 100:.2f}% of the samples.")
print("\n")


Both models are correct on 879 out of 1908 samples.
Both models are correct on 46.07% of the samples.


LLM is correct and DeBERTa_v3 is incorrect on 994 out of 1908 samples.
LLM is correct and DeBERTa_v3 is incorrect on 52.10% of the samples.


DeBERTa_v3 is correct and LLM is incorrect on 0 out of 1908 samples.
DeBERTa_v3 is correct and LLM is incorrect on 0.00% of the samples.


Both models are incorrect on 27 out of 1908 samples.
Both models are incorrect on 1.42% of the samples.


